# Export surface energy flux variables
- This script is used to export surface energy flux variables;
- Simulations: CNTL, WRF_LCZ, LI_LCZ, CESM_LCZ;

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cftime
import string
import matplotlib.dates as mdates

In [2]:
home_path = '/gws/nopw/j04/duicv/yuansun/'
archive = '/0_lcz_sp/archive/'
output_dir = home_path + '0_lcz_sp/output_analysis/sp/anthro/'    
psites = ["AU-Preston","AU-SurreyHills","CA-Sunset","FI-Kumpula","FI-Torni",
          "FR-Capitole","GR-HECKOR","JP-Yoyogi","KR-Jungnang","KR-Ochang",
          "MX-Escandon","NL-Amsterdam","PL-Lipowa","PL-Narutowicza","SG-TelokKurau06",
          "UK-KingsCollege","UK-Swindon","US-Baltimore","US-Minneapolis1","US-Minneapolis2",
          "US-WestPhoenix"]
msites = ["AU-Pre","AU-Sur","CA-Sun","FI-Kum","FI-Tor",
          "FR-Cap","GR-HEC","JP-Yoy","KR-Jun","KR-Och",
          "MX-Esc","NL-Ams","PL-Lip","PL-Nar","SG-Tel",
          "UK-Kin","UK-Swi","US-Bal","US-Mi1","US-Mi2",
          "US-Wes"]
START= ['1993', '1994', '2002', '2000', '2000',
        '1994', '2009', '2006', '2007', '2005', 
        '2001', '2009', '1998', '1998', '1996', 
        '2002', '2001', '1992', '1996', '1996', 
        '2001']
END = ['2004', '2004', '2016', '2013', '2013',
       '2005', '2020', '2020', '2019', '2017', 
       '2012', '2020',  '2012', '2012', '2007', 
       '2014', '2013',  '2007', '2009', '2009',   
       '2013']
var0 = ['LWdown', 'SWdown']
var1 = ['LWdown', 'SWdown','LWup', 'SWup', 'Qh', 'Qle', 'Qtau']
var2 = ['FIRE_U', 'FSR', 'FSH_U', 'EFLX_LH_TOT_U', 'TAUX']
zone = [10, 10, -8, 2, 2, 
        1, 2, 9, 9, 9, 
        -6, 1, 1, 1, 8, 
        0, 0, -5, -6, -6, 
        -7]
start_date = ['2003-08-12T03:30:00', '2004-02-23T05:00:00', '2012-01-01T00:00:00', '2010-12-31T22:30:00', '2010-12-31T22:30:00',
              '2004-02-20T00:30:00', '2019-06-30T22:00:00', '2016-03-31T15:00:00', '2017-01-24T16:00:00', '2015-06-07T15:00:00',
              '2011-06-01T17:00:00', '2019-01-01T00:00:00', '2008-01-01T00:00:00', '2008-01-01T00:00:00', '2006-04-30T16:30:00',
              '2012-04-04T00:00:00', '2011-05-11T19:00:00', '2002-01-01T05:00:00', '2006-06-01T18:00:00', '2006-06-01T18:00:00',
              '2011-12-16T18:30:00']
end_date = ['2004-11-28T11:30:00', '2004-07-19T22:00:00', '2016-12-31T22:00:00', '2013-12-31T20:30:00', '2013-12-31T20:30:00',
            '2005-02-28T22:30:00', '2020-06-30T20:00:00', '2020-03-31T11:00:00', '2019-04-29T05:00:00', '2017-07-26T00:00:00',
            '2012-09-13T12:30:00', '2020-10-13T08:30:00', '2012-12-31T20:00:00', '2012-12-31T20:00:00', '2007-03-31T14:30:00',
            '2013-12-31T21:00:00', '2013-04-25T08:00:00', '2007-01-01T01:00:00', '2009-05-29T11:30:00', '2008-01-01T00:00:00',
            '2013-01-01T05:00:00']
mid_date = ['2004-01-01 00:00:00', '2004-05-01 00:00:00', '2014-01-01 00:00:00', '2012-01-01 00:00:00', '2012-01-01 00:00:00',
            '2004-09-01T00:00:00', '2020-01-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2016-01-01T00:00:00',
            '2012-01-01T00:00:00', '2020-01-01T00:00:00', '2010-01-01T00:00:00', '2010-01-01T00:00:00', '2007-01-01T00:00:00',
            '2013-01-01T00:00:00', '2012-01-01T00:00:00', '2004-01-01T00:00:00', '2008-01-01T00:00:00', '2009-05-29T11:30:00',
            '2012-06-01T00:00:00']

sequence = ['Rn','SWup', 'LWup', 'Qh', 'Qle', 'Qtau']
entity = ['obs', 'def', 'wrflcz', 'lilcz']
label1 = ['Observation: mean', 'CLMU5 with default scheme (S-CNTL): mean', 'CLMU5 with WRF\'s LCZ scheme (S-WRF_LCZ): mean', 'CLMU5 with LI\'s LCZ scheme (S-LI_LCZ): mean']
label2 = ['Observation: 5-95% range', 'CLMU5 with default scheme (S-CNTL): 5-95% range', 'CLMU5 with WRF\'s LCZ scheme (S-WRF_LCZ): 5-95% range', 'CLMU5 with LI\'s LCZ scheme (S-LI_LCZ): 5-95% range']
lc0 = '#083550'
lc1 = '#006bac'
lc2 = '#3da4e3'
lc3 = '#6c64d4'
lc = [lc0, lc1, lc2, lc3]
alpha = [0.2, 0.15, 0.1, 0.1]
labelfont = 6
linewidth = 0.5
labelcolor = '#6b6b6b'
padding = 4
metric = ['7-day mean', 'Diurnal mean']
pad = 1
xwidth = 0.5
length = 2
handletextpad = 0.5
model_lable2 = ['def', 'wrflcz', 'lilcz', 'base']
labels = ['CNTL', 'WRF_LCZ', 'LI_LCZ', 'CESM_LCZ']

In [3]:
def calculate_metrics(observed_valid, *simulated):
    metrics = []
    for sim_data_valid in simulated:
        # Remove NaN values from observed and simulated data
        # Standard deviation
        std = float(np.std(sim_data_valid))
        sdev = std/float(np.std(observed_valid))
        # Correlation coefficient
        coef = float(xr.corr(xr.DataArray(observed_valid), xr.DataArray(sim_data_valid)).values)
        metrics.append((sdev, coef))
    return metrics

In [4]:
results = []
for sitesequence in range(len(psites)):
#for sitesequence in [10]:
    if sitesequence in [3, 4, 7, 8, 9, 12, 13, 14]:
        var_saved = ['LWdown', 'SWdown','LWup', 'SWup', 'Qh', 'Qle']
    elif sitesequence in [10]:
        var_saved = ['LWdown', 'SWdown','SWup', 'Qh', 'Qle', 'Qtau']
    else:    
        var_saved = var1 
    
    GRIDNAME = psites[sitesequence]
    GRIDNAME_single = msites[sitesequence]
    START_year = START[sitesequence]
    time_zone = zone[sitesequence] 
    start_time = start_date[sitesequence]
    end_time = end_date[sitesequence]
    mid_time = mid_date[sitesequence]
    
    datm = home_path + '0_lcz_sp/UrbanPlumber/datm_files/'+ GRIDNAME_single + '/CLM1PT_data/CTSM_DATM_' + GRIDNAME_single + '_' + START[sitesequence] + '-' + END[sitesequence] + '.nc'
    ds_datm = xr.open_dataset(datm)
    
    observation = home_path + '0_lcz_sp/UrbanPlumber/Urban-PLUMBER_FullCollection_v1/' + GRIDNAME + '/timeseries/'+ GRIDNAME + '_raw_observations_v1.nc'
    ds_ob = xr.open_dataset(observation)

    keith = home_path + archive + '0sp/' + GRIDNAME_single + '_def/lnd/hist/' + GRIDNAME_single + '_def.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_ko = xr.open_dataset(keith)
    
    wrf = home_path + archive + '0sp/' + GRIDNAME_single + '_wrf_all_new/lnd/hist/' + GRIDNAME_single + '_wrf_all_new.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_wrf = xr.open_dataset(wrf)
    
    li = home_path + archive + '0sp/' + GRIDNAME_single + '_li_all/lnd/hist/' + GRIDNAME_single + '_li_all.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_li = xr.open_dataset(li)
    
    base = home_path + archive + '0sp/' + GRIDNAME_single + '_cesmlcz/lnd/hist/' + GRIDNAME_single + '_cesmlcz.clm2.h0.' + START_year +'-01-01-00000.nc'
    ds_base = xr.open_dataset(base)

    ds_wrf['time']=ds_wrf['time'].dt.round('min')
    ds_wrf['time']=ds_wrf['time'].dt.ceil('min')
    ds_li['time']=ds_li['time'].dt.round('min')
    ds_li['time']=ds_li['time'].dt.ceil('min')
    ds_ko['time']=ds_ko['time'].dt.round('min')
    ds_ko['time']=ds_ko['time'].dt.ceil('min')
    ds_base['time']=ds_base['time'].dt.round('min')
    ds_base['time']=ds_base['time'].dt.ceil('min')
    
    sliced_ds_datm = ds_datm.sel(time=slice(start_time, end_time))[var0]
    df_datm = sliced_ds_datm.to_dataframe().reset_index()

    sliced_ds_ko = ds_ko.sel(time=slice(start_time, end_time))[var2]
    df_ko = sliced_ds_ko.to_dataframe().reset_index()
    df_ko.drop(columns=['lndgrid'], inplace=True)
    df_ko = df_ko.rename(columns={'FIRE_U': 'LWup_def', 'FSR': 'SWup_def','FSH_U': 'Qh_def', 'EFLX_LH_TOT_U': 'Qle_def', 'TAUX': 'Qtau_def'})
    df_ko['Qtau_def'] = - df_ko['Qtau_def']
    df_ko['Rn_def'] = df_datm['LWdown'] + df_datm['SWdown'] - df_ko['SWup_def'] - df_ko['LWup_def']
    
    sliced_ds_wrf = ds_wrf.sel(time=slice(start_time, end_time))[var2]
    df_wrf = sliced_ds_wrf.to_dataframe().reset_index()
    df_wrf.drop(columns=['lndgrid'], inplace=True)
    df_wrf = df_wrf.rename(columns={'FIRE_U': 'LWup_wrflcz', 'FSR': 'SWup_wrflcz','FSH_U': 'Qh_wrflcz', 'EFLX_LH_TOT_U': 'Qle_wrflcz', 'TAUX': 'Qtau_wrflcz'})
    df_wrf['Qtau_wrflcz'] = - df_wrf['Qtau_wrflcz']
    df_wrf['Rn_wrflcz'] = df_datm['LWdown'] + df_datm['SWdown'] - df_wrf['SWup_wrflcz'] - df_wrf['LWup_wrflcz']
    
    sliced_ds_li = ds_li.sel(time=slice(start_time, end_time))[var2]
    df_li = sliced_ds_li.to_dataframe().reset_index()
    df_li.drop(columns=['lndgrid'], inplace=True)
    df_li = df_li.rename(columns={'FIRE_U': 'LWup_lilcz', 'FSR': 'SWup_lilcz','FSH_U': 'Qh_lilcz', 'EFLX_LH_TOT_U': 'Qle_lilcz', 'TAUX': 'Qtau_lilcz'})
    df_li['Qtau_lilcz'] = - df_li['Qtau_lilcz']
    df_li['Rn_lilcz'] = df_datm['LWdown'] + df_datm['SWdown'] - df_li['SWup_lilcz'] - df_li['LWup_lilcz']
    
    sliced_ds_base = ds_base.sel(time=slice(start_time, end_time))[var2]
    df_base = sliced_ds_base.to_dataframe().reset_index()
    df_base.drop(columns=['lndgrid'], inplace=True)
    df_base = df_base.rename(columns={'FIRE_U': 'LWup_base', 'FSR': 'SWup_base','FSH_U': 'Qh_base', 'EFLX_LH_TOT_U': 'Qle_base', 'TAUX': 'Qtau_base'})
    df_base['Qtau_base'] = - df_base['Qtau_base']
    df_base['Rn_base'] = df_datm['LWdown'] + df_datm['SWdown'] - df_base['SWup_base'] - df_base['LWup_base']
    
    if sitesequence in [3, 4, 7, 8, 9, 12, 13, 14]:
        sliced_ds_ob = ds_ob.sel(time=slice(start_time, end_time))[['LWdown', 'SWdown','LWup', 'SWup', 'Qh', 'Qle']]
        df_ob = sliced_ds_ob.to_dataframe().reset_index()
        df_ob = df_ob.rename(columns={'LWdown': 'LWdown_obs', 
                                      'SWdown': 'SWdown_obs',
                                      'LWup': 'LWup_obs', 
                                      'SWup': 'SWup_obs',
                                      'Qh': 'Qh_obs', 
                                      'Qle': 'Qle_obs'})
        df_ob['Qtau_obs'] = np.nan
        
    else: 
        sliced_ds_ob = ds_ob.sel(time=slice(start_time, end_time))[var1]
        df_ob = sliced_ds_ob.to_dataframe().reset_index()
        df_ob = df_ob.rename(columns={'LWdown': 'LWdown_obs', 
                                      'SWdown': 'SWdown_obs',
                                      'LWup': 'LWup_obs', 
                                      'SWup': 'SWup_obs',
                                      'Qh': 'Qh_obs', 
                                      'Qle': 'Qle_obs', 
                                      'Qtau': 'Qtau_obs'})
    df_ob['Rn_obs'] = df_ob['LWdown_obs'] + df_ob['SWdown_obs']-df_ob['SWup_obs']-df_ob['LWup_obs']
    
    df = pd.merge(df_ob, df_ko, on='time').merge(df_wrf, on='time').merge(df_li, on='time').merge(df_base, on='time')
    df.loc[df['LWup_obs'].isna(), ['LWup_base', 'LWup_wrflcz', 'LWup_lilcz','LWup_def',
                                  'Rn_base','Rn_obs', 'Rn_wrflcz', 'Rn_lilcz', 'Rn_def']] = np.nan
    df.loc[df['SWup_obs'].isna(), ['SWup_base', 'SWup_wrflcz', 'SWup_lilcz', 'SWup_def',
                                  'Rn_base', 'Rn_obs', 'Rn_wrflcz', 'Rn_lilcz', 'Rn_def']] = np.nan
    df.loc[df['Qle_obs'].isna(), ['Qle_base', 'Qle_wrflcz', 'Qle_lilcz','Qle_def']] = np.nan
    df.loc[df['Qh_obs'].isna(), ['Qh_base', 'Qh_wrflcz', 'Qh_lilcz','Qh_def']] = np.nan
    df.loc[df['Qtau_obs'].isna(), ['Qtau_base', 'Qtau_wrflcz', 'Qtau_lilcz','Qtau_def']] = np.nan
    #df_total = pd.concat([df, df_total], ignore_index=True)
    for variable in sequence:
        for j, model in enumerate(model_lable2):
            model_key = f'{variable}_{model}'
            obs_key = f'{variable}_obs'
            if model_key in df:
               mean_sdev = float(np.std(df[model_key])) / float(np.std(df[obs_key]))
               mean_coef = float(xr.corr(xr.DataArray(df[model_key]), xr.DataArray(df[obs_key])))
               results.append({'Site': psites[sitesequence],'Variable': variable, 'Model':labels[j], 'mean_sdev': mean_sdev, 'mean_coef': mean_coef})
results_df = pd.DataFrame(results)
results_df.to_csv(output_dir + 'flux_bysite.csv', index=False)      
results_df           
  

/home/users/yuan1239/.conda/envs/conda_env/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/users/yuan1239/.conda/envs/conda_env/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/users/yuan1239/.conda/envs/conda_env/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/users/yuan1239/.conda/envs/conda_env/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/users/yuan1239/.conda/envs/conda_env/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:2035: Run